In [1]:
import pandas as pd 
import ast 

In [2]:
file_path = "/Users/flint/Data/recipe/italian/gz_recipe.csv"

In [3]:
data = pd.read_csv(file_path, index_col=0)
data.head()

,Nome,Categoria,Link,Persone/Pezzi,Ingredienti,Steps
0,Tiramisù,Dolci,https://ricette.giallozafferano.it/Tiramisu.html,8,"[['Mascarpone', '750g'], ['Uova', '260g'], ['S...",Per preparare il tiramisù preparate il caffé c...
1,Cookies,Dolci,https://ricette.giallozafferano.it/Cookies.html,12,"[['Farina 00', '195g'], ['Burro', '100g'], ['B...","Per preparare i cookies, assicuratevi che il b..."
2,Pancake allo sciroppo d'acero,Dolci,https://ricette.giallozafferano.it/Pancakes-al...,4,"[['Burro', '25g'], ['Farina 00', '125g'], ['Uo...",Iniziamo la preparazione dei pancake fondendo ...
3,Crema al mascarpone,Dolci,https://ricette.giallozafferano.it/Crema-al-ma...,4,"[['Mascarpone', '500g'], ['Zucchero', '125g'],...",Per preparare la crema al mascarpone versate i...
4,Crepe dolci e salate (ricetta base),Dolci,https://ricette.giallozafferano.it/Crepes-dolc...,15,"[['Uova', '3'], ['Farina 00', '250g'], ['Latte...",Per preparare le crepe dolci e salate iniziate...


In [15]:
raw_ingredients = [ast.literal_eval(x) for x in data['Ingredienti'].values]
ingredient_table = []
for i, name in enumerate(data['Nome']):
    ingredients_list = raw_ingredients[i]
    for ingredient, quantity in ingredients_list:
        try:
            qta = int(quantity[:-1])
        except Exception:
            qta = 1
        ingredient_table.append({
            'ricetta': i,
            'ingrediente': ingredient,
            'qta': qta
        })
ingredients = pd.DataFrame(ingredient_table)

In [16]:
ingredients

,ricetta,ingrediente,qta
0,0,Mascarpone,750
1,0,Uova,260
2,0,Savoiardi,250
3,0,Zucchero,120
4,0,Caffè,300
...,...,...,...
65105,5938,Olio extravergine d'oliva,1
65106,5938,Sale fino,1
65107,5938,Pepe nero,1
65108,5938,Basilico,1


## Creazione della matrice documenti $\times$ termini (in questo caso ricetta $\times$ ingrediente)

In [14]:
import numpy as np 

In [23]:
vocabulary = sorted(ingredients.ingrediente.unique())
print(len(vocabulary), vocabulary[:4])
ingredient2id = dict([(x, i) for i, x in enumerate(vocabulary)])
id2ingredient = dict([(i, x) for i, x in enumerate(vocabulary)])
recipe2id = dict([(x, i) for i, x in enumerate(data['Nome'].values)])
id2recipe = dict([(i, x) for i, x in enumerate(data['Nome'].values)])

1641 ['Abbacchio', 'Acai in polvere', 'Acciughe (alici)', "Acciughe sott'olio"]


In [24]:
matrix = np.zeros((data.shape[0], len(vocabulary)))
for i, row in ingredients.iterrows():
    recipe = row['ricetta']
    ingredient = row['ingrediente']
    qta = row['qta']
    matrix[recipe, ingredient2id[ingredient]] = qta 

In [ ]:
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
test = 'Crema al mascarpone'
test_vector = pd.Series(matrix[recipe2id[test]], index=vocabulary)
text_vector.sort_values(ascending=False)

Mascarpone                     500.0
Zucchero                       125.0
Acqua                           50.0
Tuorli                           1.0
Cacao amaro in polvere           1.0
                               ...  
Fichi                            0.0
Fettuccine all'uovo              0.0
Fettuccine 5 cereali e uova      0.0
Fettuccine                       0.0
zenzero candito                  0.0
Length: 1641, dtype: float64

### Similarità fra ricette

In [31]:
from sklearn.metrics.pairwise import pairwise_distances

In [32]:
sigma = pairwise_distances(matrix, matrix, metric='euclidean')

In [37]:
test = 'Focaccia col formaggio (di Recco)'
test_id = recipe2id[test]
sigma[test_id]

array([380802.11929163, 380801.21858524, 380801.14701639, ...,
       380801.44768501, 380801.01658215, 380801.8933632 ])

In [38]:
most_similar = pd.Series(sigma[test_id], index=data['Nome']).sort_values(ascending=True)
most_similar

Nome
Focaccia col formaggio (di Recco)                                                          0.000000e+00
Panini delle feste                                                                         3.800511e+05
Stollen                                                                                    3.801013e+05
Focaccia con alici, fiori di zucca e burrata                                               3.802009e+05
Jerusalem bagel                                                                            3.802011e+05
                                                                                               ...     
Sigari con caprino e miele al rosmarino                                                    4.522994e+05
Maritozzi golosi                                                                           4.976602e+05
Gnocchi coi Useleti (Gnocchi con gli involtini di carne)                                   5.898818e+05
Cestini di pasta fillo con cremoso di cioccolato bianco e c

## Manipolazione del testo della ricetta
Invece degli ingredienti, utilizziamo il testo tokenizzato

In [40]:
from tqdm.notebook import tqdm
from transformers import BertTokenizer

In [41]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-uncased")

In [45]:
recipes_text = data['Steps'].values 
vocabulary = set()
recipes_tokens = []
for text in tqdm(recipes_text):
    try:
        tokens = tokenizer.tokenize(text)
        vocabulary = vocabulary.union(set(tokens))
        recipes_tokens.append(tokens)
    except TypeError:
        recipes_tokens.append([])

  0%|          | 0/5939 [00:00<?, ?it/s]

In [50]:
vocabulary = sorted(list(vocabulary))
word2id = dict(zip(vocabulary, range(len(vocabulary))))
id2word = dict(zip(range(len(vocabulary)), vocabulary))
X = np.zeros((data.shape[0], len(vocabulary)))
for recipe, tokens in enumerate(recipes_tokens):
    for token in tokens:
        X[recipe, word2id[token]] += 1

In [52]:
test = 'Focaccia col formaggio (di Recco)'
test_id = recipe2id[test]
bow = pd.Series(X[test_id], index=vocabulary)

In [56]:
bow.sort_values(ascending=False).head(10)

,      25.0
.      24.0
di     21.0
e      20.0
la     18.0
con    15.0
a      13.0
una    13.0
in     12.0
per    12.0
dtype: float64

## Similarità lessicale

In [57]:
sigma_words = pairwise_distances(X, X, metric='euclidean')

In [58]:
most_similar = pd.Series(sigma_words[test_id], index=data['Nome']).sort_values(ascending=True)
most_similar.sort_values()

Nome
Focaccia col formaggio (di Recco)      0.000000
Pizza Margherita                      41.521079
Cannelloni ricotta e spinaci          41.653331
Tagliatelle ai funghi porcini         42.213742
Cannoli siciliani                     42.602817
                                        ...    
Tiramisù goloso                       73.803794
Casetta di pan di spezie              79.686887
Shish Kebab di tacchino               82.462113
Torta libro                           86.596767
Torta Barbie                         111.575087
Length: 5939, dtype: float64